### Configuration

In [ ]:
%%configure
{
    "conf": {
            "spark.jars":"hdfs:///apps/hudi/lib/hudi-spark-bundle.jar,hdfs:///apps/hudi/lib/spark-avro.jar",
            "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
            "spark.sql.hive.convertMetastoreParquet":"false"
    }
}

In [2]:
from pyspark.sql.functions import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1635772718416_0042,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Create a DataFrame

In [3]:
inputDF = spark.createDataFrame(
    [
        ("100", "2015-01-01", "2015-01-01T13:51:39.340396Z"),
        ("101", "2015-01-01", "2015-01-01T12:14:58.597216Z"),
        ("102", "2015-01-01", "2015-01-01T13:51:40.417052Z"),
        ("103", "2015-01-01", "2015-01-01T13:51:40.519832Z"),
        ("104", "2015-01-02", "2015-01-01T12:15:00.512679Z"),
        ("105", "2015-01-02", "2015-01-01T13:51:42.248818Z"),
    ],
    ["id", "creation_date", "last_update_time"]
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
inputDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-------------+--------------------+
| id|creation_date|    last_update_time|
+---+-------------+--------------------+
|100|   2015-01-01|2015-01-01T13:51:...|
|101|   2015-01-01|2015-01-01T12:14:...|
|102|   2015-01-01|2015-01-01T13:51:...|
|103|   2015-01-01|2015-01-01T13:51:...|
|104|   2015-01-02|2015-01-01T12:15:...|
|105|   2015-01-02|2015-01-01T13:51:...|
+---+-------------+--------------------+

### Write to S3 via. Hudi

In [5]:
# Specify common DataSourceWriteOptions in the single hudiOptions variable
hudiOptions = {
    'hoodie.table.name': 'my_hudi_table',
    'hoodie.datasource.write.recordkey.field': 'id',
    'hoodie.datasource.write.partitionpath.field': 'creation_date',
    'hoodie.datasource.write.precombine.field': 'last_update_time',
    'hoodie.datasource.hive_sync.enable': 'true',
    'hoodie.datasource.hive_sync.table': 'my_hudi_table',
    'hoodie.datasource.hive_sync.partition_fields': 'creation_date',
    'hoodie.datasource.hive_sync.partition_extractor_class': 'org.apache.hudi.hive.MultiPartKeysValueExtractor'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Write a DataFrame as a Hudi dataset
inputDF.write.format('org.apache.hudi').option('hoodie.datasource.write.operation', 'insert').options(**hudiOptions).mode('overwrite').save('s3://hudi-sharkech/myhudidataset/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Upsert data

In [7]:
# Create a new DataFrame from the first row of inputDF with a different creation_date value
updateDF = inputDF.limit(1).withColumn('creation_date', lit('2016-02-02'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
updateDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-------------+--------------------+
| id|creation_date|    last_update_time|
+---+-------------+--------------------+
|100|   2016-02-02|2015-01-01T13:51:...|
+---+-------------+--------------------+

In [9]:
updateDF.write.format('org.apache.hudi').option('hoodie.datasource.write.operation', 'upsert').options(**hudiOptions).mode('append').save('s3://hudi-sharkech/myhudidataset/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Check that the upsert worked
snapshotQueryDF = spark.read.format('org.apache.hudi').load('s3://hudi-sharkech/myhudidataset' + '/*/*')

snapshotQueryDF.select("id", "creation_date", "last_update_time").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-------------+--------------------+
| id|creation_date|    last_update_time|
+---+-------------+--------------------+
|100|   2016-02-02|2015-01-01T13:51:...|
|100|   2015-01-01|2015-01-01T13:51:...|
|101|   2015-01-01|2015-01-01T12:14:...|
|102|   2015-01-01|2015-01-01T13:51:...|
|103|   2015-01-01|2015-01-01T13:51:...|
|104|   2015-01-02|2015-01-01T12:15:...|
|105|   2015-01-02|2015-01-01T13:51:...|
+---+-------------+--------------------+

### Delete a record

In [11]:
updateDF.write.format('org.apache.hudi').option('hoodie.datasource.write.operation', 'upsert').option('hoodie.datasource.write.payload.class', 'org.apache.hudi.common.model.EmptyHoodieRecordPayload').options(**hudiOptions).mode('append').save('s3://hudi-sharkech/myhudidataset/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Check that the delete worked
snapshotQueryDF_v2 = spark.read.format('org.apache.hudi').load('s3://hudi-sharkech/myhudidataset' + '/*/*')

snapshotQueryDF_v2.select("id", "creation_date", "last_update_time").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-------------+--------------------+
| id|creation_date|    last_update_time|
+---+-------------+--------------------+
|100|   2015-01-01|2015-01-01T13:51:...|
|101|   2015-01-01|2015-01-01T12:14:...|
|102|   2015-01-01|2015-01-01T13:51:...|
|103|   2015-01-01|2015-01-01T13:51:...|
|104|   2015-01-02|2015-01-01T12:15:...|
|105|   2015-01-02|2015-01-01T13:51:...|
+---+-------------+--------------------+

### Incremental queries

In [16]:
readOptions = {
  'hoodie.datasource.query.type': 'incremental',
  'hoodie.datasource.read.begin.instanttime': "2016-01-01",
}

incQueryDF = spark.read.format('org.apache.hudi').options(**readOptions).load('s3://hudi-sharkech/myhudidataset/')
    
incQueryDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+------------------+----------------------+--------------------+---+-------------+--------------------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name| id|creation_date|    last_update_time|
+-------------------+--------------------+------------------+----------------------+--------------------+---+-------------+--------------------+
|     20211101160626|  20211101160626_0_2|               100|            2015-01-01|66fbfa92-fcff-41d...|100|   2015-01-01|2015-01-01T13:51:...|
|     20211101160626|  20211101160626_0_3|               101|            2015-01-01|66fbfa92-fcff-41d...|101|   2015-01-01|2015-01-01T12:14:...|
|     20211101160626|  20211101160626_0_5|               102|            2015-01-01|66fbfa92-fcff-41d...|102|   2015-01-01|2015-01-01T13:51:...|
|     20211101160626|  20211101160626_0_6|               103|            2015-01-01|66fbfa92-fcff-41d...|103|   2015-01-01|2015-01